In [63]:
from crewai import Crew, Task, Agent, LLM
from crewai_tools import RagTool
from dotenv import load_dotenv
import os
from langfuse import Langfuse
from openinference.instrumentation.crewai import CrewAIInstrumentor
from openinference.instrumentation.litellm import LiteLLMInstrumentor
 
CrewAIInstrumentor().instrument(skip_dep_check=True)
LiteLLMInstrumentor().instrument()

load_dotenv(override=True)

import warnings
warnings.filterwarnings('ignore')

2025-07-13 08:47:19,850 - 8389430272 - instrumentor.py-instrumentor:103 - WARNING: Attempting to instrument while already instrumented
2025-07-13 08:47:19,852 - 8389430272 - instrumentor.py-instrumentor:103 - WARNING: Attempting to instrument while already instrumented


In [64]:
langfuse = Langfuse()

if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


In [65]:
llm = LLM(model="azure/gpt-4.1", max_tokens=1024)

In [66]:
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": "gpt-4.1-mini",
        }
    },
    "embedding_model": {
        "provider": "azure_openai",
        "config": {
            "model": "text-embedding-ada-002"
        }
    }
}

In [67]:
rag_tool = RagTool(config=config,  
    chunk_size=1200,       
    chunk_overlap=200,     
)

rag_tool.add("./data/gold-hospital-and-premium-extras.pdf", data_type="pdf_file")

# Define the Insurance Agent

In [68]:
insurance_agent = Agent(
    role="Senior Insurance Coverage Assistant", 
    goal="Determine whether something is covered or not",
    backstory="You are an expert insurance agent designed to assist with coverage queries",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[rag_tool], 
    max_retry_limit=5
)

In [72]:
task1 = Task(
    description='Thời gian chờ đợi để phục hồi chức năng là bao lâu?',
    expected_output = "Một phản hồi toàn diện cho câu hỏi của người dùng",
    agent=insurance_agent
)

In [73]:
crew = Crew(agents=[insurance_agent], tasks=[task1], verbose=True)

with langfuse.start_as_current_span(name="crewai-insurance-trace"):
    result = crew.kickoff()
    print(result)

langfuse.flush()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4a8f353b-c804-466a-8761-463d2881689c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Task: Thời gian chờ đợi để phục hồi chức năng là bao lâu?                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Usage Failed                                                                                              │
│  Name: Knowledge base                                                                                           │
│  Error: Arguments validation failed: 1 validation error for RagToolSchema                                       │
│  query                                                                                                          │
│    Field required [type=missing, input_value={'description': 'thời g...9099', 'metadata': {}}}},                │
│  input_type=dict]                                                                                               │
│      For further information visit https://errors.pydantic.dev/2.11/v/missing                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Usage Failed                                                                                              │
│  Name: Knowledge base                                                                                           │
│  Error: Arguments validation failed: 1 validation error for RagToolSchema                                       │
│  query                                                                                                          │
│    Field required [type=missing, input_value={'description': 'thời g...9099', 'metadata': {}}}},                │
│  input_type=dict]                                                                                               │
│      For further information visit https://errors.pydantic.dev/2.11/v/missing                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Usage Failed                                                                                              │
│  Name: Knowledge base                                                                                           │
│  Error: Arguments validation failed: 1 validation error for RagToolSchema                                       │
│  query                                                                                                          │
│    Field required [type=missing, input_value={'description': 'thời g...9099', 'metadata': {}}}},                │
│  input_type=dict]                                                                                               │
│      For further information visit https://errors.pydantic.dev/2.11/v/missing                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  CLINICAL CATEGORIES WAITING PERIOD GOLD Rehabilitation 2 months 4 Hospital psychiatric services 2 months 4     │
│  Palliative care 2 months 4 Brain and nervous system 2 months 4 Eye (not cataracts) 2 months 4 Ear, nose and    │
│  throat 2 months 4 Tonsils, adenoids and grommets 2 months 4 Bone, joint and muscle 2 months 4 Joint            │
│  reconstructions 2 months 4 Kidney and bladder 2 months 4 Male reproductive system 2 months 4 Digestive system  │
│  2 months 4 Hernia and appendix 2 months 4 Gastrointestinal endoscopy 2 months 4 Gynaecology 2 months 4         │
│  Miscarriage and termination of pregnancy 2 months 4 Chemotherapy, radiotherapy and immunotherapy for cancer 2  │
│  months 4 Pain management 2 months 4 Skin 2 months 4 Breast surgery (medically necessary) 2 months 4 CLINICAL   │
│  CATEGORIES WAITING PERIOD GOLD Diabetes management (excluding insulin pumps) 2 months 4 Heart and vascular     │
│  system 2 months 4 Lung and chest 2 months 4 Blood 2 months 4 Back, neck and spine 2 months 4 Plastic and       │
│  reconstructive surgery (medically                                                                              │
│                                                                                                                 │
│  Health Programs & Support Our Health and Hospital Substitution programs are designed to support our members    │
│  on their health journey or on their road to recovery. We have a number of programs to suit a variety of        │
│  health and recovery needs. HEALTH PROGRAMS Our health programs are designed to help you keep on top of your    │
│  health and live a healthier life. We have a range of health programs to help you manage a number of different  │
│  health conditions. The waiting period to recieve health programs is 2 months of continuous cover. HOSPITAL     │
│  SUBSTITUTION PROGRAMS* Our Hospital substitution programs allow you and/or your family members to recover in   │
│  the comfort of your own home with a range of in home hospital treatments, so you can have the choice of        │
│  treatments that suit your needs. You’ll need to have served your 2-month waiting period and have a referral    │
│  from your treating doctor to be eligible. Please note, anything that is a pre-existing condition will have a   │
│  12-month waiting p...                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Thời gian chờ đợi để được phục hồi chức năng (rehabilitation) trong bảo hiểm là 2 tháng tính từ khi bạn bắt    │
│  đầu tham gia bảo hiểm hoặc chuyển đổi gói bảo hiểm. Điều này nghĩa là bạn cần duy trì đóng phí bảo hiểm liên   │
│  tục trong 2 tháng trước khi được hưởng quyền lợi phục hồi chức năng.                                           │
│                                                                                                                 │
│  Tuy nhiên, có một lưu ý quan trọng: Nếu việc phục hồi chức năng liên quan đến một tình trạng bệnh đã có từ     │
│  trước (pre-existing condition), thời gian chờ đợi đối với các mục phục hồi chức năng, dịch vụ tâm thần tại     │
│  bệnh viện, chăm sóc giảm nhẹ (palliative care) và dịch vụ xe cứu thương vẫn là 2 tháng, không áp dụng thời     │
│  gian chờ 12 tháng như những quyền lợi khác về tình trạng bệnh có từ trước.                                     │
│                                                                                                                 │
│  Nếu bạn tham gia các chương trình Health Programs & Support hoặc Hospital Substitution Programs (chương trình  │
│  phục hồi và thay thế điều trị tại nhà), bạn cũng cần đủ 2 tháng đóng bảo hiểm liên tục, đồng thời phải có      │
│  giấy giới thiệu từ bác sĩ điều trị.                                                                            │
│                                                                                                                 │
│  Lưu ý, đây là quy định chung, có thể thay đổi tùy theo gói bảo hiểm cụ thể hoặc nhà cung cấp. Vì vậy, bạn nên  │
│  liên hệ trực tiếp công ty bảo hiểm để kiểm tra quyền lợi chi tiết trước khi nhập viện hoặc sử dụng dịch vụ     │
│  phục hồi chức năng nhằm được tư vấn chính xác nhất cho trường hợp của mình.                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1e97abbb-afa9-4058-b81b-8b7a0fbf20e4                                                                     │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4a8f353b-c804-466a-8761-463d2881689c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Thời gian chờ đợi để được phục hồi chức năng (rehabilitation) trong bảo hiểm là 2 tháng tính từ  │
│  khi bạn bắt đầu tham gia bảo hiểm hoặc chuyển đổi gói bảo hiểm. Điều này nghĩa là bạn cần duy trì đóng phí     │
│  bảo hiểm liên tục trong 2 tháng trước khi được hưởng quyền lợi phục hồi chức năng.                             │
│                                                                                                                 │
│  Tuy nhiên, có một lưu ý quan trọng: Nếu việc phục hồi chức năng liên quan đến một tình trạng bệnh đã có từ     │
│  trước (pre-existing condition), thời gian chờ đợi đối với các mục phục hồi chức năng, dịch vụ tâm thần tại     │
│  bệnh viện, chăm sóc giảm nhẹ (palliative care) và dịch vụ xe cứu thương vẫn là 2 tháng, không áp dụng thời     │
│  gian chờ 12 tháng như những quyền lợi khác về tình trạng bệnh có từ trước.                                     │
│                                                                                                                 │
│  Nếu bạn tham gia các chương trình Health Programs & Support hoặc Hospital Substitution Programs (chương trình  │
│  phục hồi và thay thế điều trị tại nhà), bạn cũng cần đủ 2 tháng đóng bảo hiểm liên tục, đồng thời phải có      │
│  giấy giới thiệu từ bác sĩ điều trị.                                                                            │
│                                                                                                                 │
│  Lưu ý, đây là quy định chung, có thể thay đổi tùy theo gói bảo hiểm cụ thể hoặc nhà cung cấp. Vì vậy, bạn nên  │
│  liên hệ trực tiếp công ty bảo hiểm để kiểm tra quyền lợi chi tiết trước khi nhập viện hoặc sử dụng dịch vụ     │
│  phục hồi chức năng nhằm được tư vấn chính xác nhất cho trường hợp của mình.                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Thời gian chờ đợi để được phục hồi chức năng (rehabilitation) trong bảo hiểm là 2 tháng tính từ khi bạn bắt đầu tham gia bảo hiểm hoặc chuyển đổi gói bảo hiểm. Điều này nghĩa là bạn cần duy trì đóng phí bảo hiểm liên tục trong 2 tháng trước khi được hưởng quyền lợi phục hồi chức năng.

Tuy nhiên, có một lưu ý quan trọng: Nếu việc phục hồi chức năng liên quan đến một tình trạng bệnh đã có từ trước (pre-existing condition), thời gian chờ đợi đối với các mục phục hồi chức năng, dịch vụ tâm thần tại bệnh viện, chăm sóc giảm nhẹ (palliative care) và dịch vụ xe cứu thương vẫn là 2 tháng, không áp dụng thời gian chờ 12 tháng như những quyền lợi khác về tình trạng bệnh có từ trước.

Nếu bạn tham gia các chương trình Health Programs & Support hoặc Hospital Substitution Programs (chương trình phục hồi và thay thế điều trị tại nhà), bạn cũng cần đủ 2 tháng đóng bảo hiểm liên tục, đồng thời phải có giấy giới thiệu từ bác sĩ điều trị.

Lưu ý, đây là quy định chung, có thể thay đổi tùy theo gói bảo 